In [1]:
import numpy as np
import copy
import pickle

from dipy.io.streamline import load_trk
from dipy.io.image import load_nifti, save_nifti

from dipy.viz import regtools
from dipy.viz import actor, window, ui
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)

from dipy.tracking import streamline

import simnibs
from simnibs import sim_struct, run_simnibs

import vtkplotter

embedWindow(verbose=True): could not load k3d module, try:
> pip install k3d      # and if necessary:
> conda install nodejs


In [2]:
# reads the tractography data in trk format
# extracts streamlines and the file header. Streamlines should be in the same coordinate system as the FA map (used later).
# input example: '/home/sofya/RNF/NOVOKOV/Diffusion/DTI/CST_6.trk'
tractography_file=input("Please, specify the file with tracts that you would like to analyse. File should be in the trk format")

streams, hdr = load_trk(tractography_file)
streams_array=np.asarray(streams)
print ('imported tractography data:'+tractography_file)

Please, specify the file with tracts that you would like to analyse. File should be in the trk format/home/sofya/RNF/NOVOKOV/Diffusion/DTI/cst_1358.trk
imported tractography data:/home/sofya/RNF/NOVOKOV/Diffusion/DTI/cst_1358.trk


In [6]:
# load T1fs_conform image that operates in the same coordinates as simnibs except for the fact the center of mesh 
# is located at the image center
# T1fs_conform image should be generated in advance during the head meshing procedure
# input example: fname_T1='/home/sofya/RNF/NOVOKOV/T1/m2m_NOVIKOV/T1fs_conform.nii.gz'

fname_T1=input("Please, specify the T1fs_conform image that has been generated during head meshing procedure")
data_T1, affine_T1 = load_nifti(fname_T1)

# load FA image in the same coordinates as tracts
# input example:fname_FA='/home/sofya/RNF/NOVOKOV/Diffusion/DTI/dti_fa.nii'
fname_FA=input("Please, specify the FA image that has been generated during head meshing procedure")
data_FA, affine_FA = load_nifti(fname_FA)

print ('loaded T1fs_conform.nii and FA images')

Please, specify the T1fs_conform image that has been generated during head meshing procedure/home/sofya/RNF/NOVOKOV/T1/m2m_NOVIKOV/T1fs_conform.nii.gz
Please, specify the FA image that has been generated during head meshing procedure/home/sofya/RNF/NOVOKOV/Diffusion/DTI/dti_fa.nii
loaded T1fs_conform.nii and FA images


In [7]:
# specify the head mesh file that is used later in simnibs to simulate induced electric field
# input example:'/media/sofya/Seagate Backup Plus Drive/NOVOKOV/T1/NOVIKOV.msh'
mesh_path=input("Please, specify the head mesh file")

last_slach=max([i for i, ltr in enumerate(mesh_path) if ltr == '/'])+1
subject_name=mesh_path[last_slach:-4]

# specify the directory where you would like to save your simulation results
# input example:'/media/sofya/Seagate Backup Plus Drive/NOVOKOV/T1/StimVis'
out_dir=input("Please, specify the directory where you would like to save your simulation results")
out_dir=out_dir+'/simulation_at_pos_'

Please, specify the head mesh file/home/sofya/Example_data/subject.msh
Please, specify the directory where you would like to save your simulation results/home/sofya/Example_data/Output


In [8]:
# Co-registration of T1fs_conform and FA images. Performed in 4 steps.
# Step 1. Calculation of the center of mass transform. Used later as starting transform.
c_of_mass = transform_centers_of_mass(data_T1, affine_T1,
                                      data_FA, affine_FA)
print ('calculated c_of_mass transformation')

# Step 2. Calculation of a 3D translation transform. Used in the next step as starting transform.
nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)
level_iters = [10000, 1000, 100]
sigmas = [3.0, 1.0, 0.0]
factors = [4, 2, 1]
affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(data_T1, data_FA, transform, params0,
                              affine_T1, affine_FA,
                              starting_affine=starting_affine)
print ('calculated 3D translation transform')

# Step 3. Calculation of a Rigid 3D transform. Used in the next step as starting transform
transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(data_T1, data_FA, transform, params0,
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)
print ('calculated Rigid 3D transform')

# Step 4. Calculation of an affine transform. Used for co-registration of T1 and FA images. 
transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(data_T1, data_FA, transform, params0,
                        affine_T1, affine_FA,
                        starting_affine=starting_affine)

print ('calculated Affine 3D transform')

calculated c_of_mass transformation
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated 3D translation transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Rigid 3D transform
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
calculated Affine 3D transform


In [9]:
identity = np.eye(4)

inv_affine_FA=np.linalg.inv(affine_FA)
inv_affine_T1=np.linalg.inv(affine_T1)
inv_affine=np.linalg.inv(affine.affine) 

# transforming streamlines to FA space
new_streams_FA=streamline.transform_streamlines(streams, inv_affine_FA)
new_streams_FA_array=np.asarray(new_streams_FA)

T1_to_FA=np.dot(inv_affine_FA, np.dot(affine.affine, affine_T1)) 
FA_to_T1=np.linalg.inv(T1_to_FA)

# transforming streamlines from FA to T1 space
new_streams_T1=streamline.transform_streamlines(new_streams_FA, FA_to_T1)
new_streams_T1_array=np.asarray(new_streams_T1)

In [10]:
# # исключительно для визуализации и в DAMDID
all_new_streams_T1 = streamline.transform_streamlines(new_streams_FA, FA_to_T1)
all_new_streams_T1_array = np.asarray(all_new_streams_T1)

new_streams_T1 = []
selected_fibers = [73, 170, 172, 208, 210, 211, 255, 261, 263, 700, 701, 773] 
for good_fiber in selected_fibers :
    new_streams_T1.append(all_new_streams_T1_array[good_fiber])

new_streams_T1_array = np.asarray(new_streams_T1)

In [11]:
# a function to compute 1st-order numerical derivative using a 3-point schema
# t - a point (index in the 'line' array) at which derivative should be computed
# line - array representing a function
# h - step between the points on the line

def my_deriv(t,line,h=1):
    if t==0:
        return (-3*line[t]+4*line[t+1]-line[t+2])/(2*h)
    elif t==len(line)-1:
        return (line[t-2]-4*line[t-1]+3*line[t])/(2*h)
    else:
        return (line[t+1]-line[t-1])/(2*h)

In [59]:
# calculating amline derivatives along the streamlines to get the local orientation of the streamlines

streams_array_derivative=copy.deepcopy(new_streams_T1_array)

print ('calculating amline derivatives')
for stream in range(len(new_streams_T1_array)):
    my_steam=new_streams_T1_array[stream]
    for t in range(len(my_steam[:,0])):
        streams_array_derivative[stream][t,0]=my_deriv(t,my_steam[:,0])
        streams_array_derivative[stream][t,1]=my_deriv(t,my_steam[:,1])
        streams_array_derivative[stream][t,2]=my_deriv(t,my_steam[:,2])
        deriv_norm=np.linalg.norm(streams_array_derivative[stream][t,:])
        streams_array_derivative[stream][t,:]=streams_array_derivative[stream][t,:]/deriv_norm

calculating amline derivatives


In [13]:
# This function is to run simulations of the induced magnetic field using simnibs software

def simulation(fnamehead, pathfem, pos_centre=[-74.296158, -10.213354, 28.307243], pos_ydir=[-74.217369, -37.293205, 20.05232], distance=4, current_change=1e6):
    # Initalize a session
    s = sim_struct.SESSION()
    # Name of head mesh
    s.fnamehead = fnamehead
    # Output folder
    s.pathfem = pathfem
    # Not to visualize results in gmsh when running simulations (else set to True)
    s.open_in_gmsh=False
    
    # Initialize a list of TMS simulations
    tmslist = s.add_tmslist()
    # Select coil. For full list of available coils, please see simnibs documentation
    tmslist.fnamecoil = 'Magstim_70mm_Fig8.nii.gz'
    
    # Initialize a coil position
    pos = tmslist.add_position()
    pos.centre = pos_centre # Place the coil over
    pos.pos_ydir = pos_ydir # Point the coil towards
    pos.distance = distance # Distance between coil and head
    pos.didt = current_change # Rate of change of current in the coil, in A/s.
    run_simnibs(s)

In [14]:
#%%cython -a
def load_elems(nodes,elems):

    import numpy as np

    elems = elems[elems[:,3]!= -1,:]
    # Computing rectangles
    tmp = nodes[elems-1,:]
    elems_min = tmp.min(axis=1)
    elems_max = tmp.max(axis=1)
    tmp = 0
    sizes = (elems_max-elems_min).max(axis=0)
    # It is the index to reduce the elements to check
    order_min = np.argsort(elems_min[:,0])
    return {"Nodes":nodes, "Elems":elems, "El_min":elems_min, "El_max":elems_max, "Sizes":sizes,"Order_min":order_min}

def get_ttrd(loaded_elems,point):
    import numpy as np
    # Just to use names I have used before
    nodes = loaded_elems["Nodes"]
    elems = loaded_elems["Elems"]
    elems_min = loaded_elems["El_min"]
    elems_max = loaded_elems["El_max"]
    sizes = loaded_elems["Sizes"]
    order_min = loaded_elems["Order_min"]
    
    # Binary search to reduce the iterating points from 4mln to around 200k.
    r = np.searchsorted(elems_min[:,0],point[0],side='left',sorter=order_min)
    l = np.searchsorted(elems_min[:,0],point[0] - sizes[0],side='right',sorter=order_min)
    # Projection the data to only these points
    e_max = elems_max[order_min[l:r]]
    e_min = elems_min[order_min[l:r]]
    
    # Checks which ttrds are possible to contain the point
    potential_ttrds = order_min[l:r][(point[0] <= e_max[:,0]) & (e_min[:,1]<= point[1]) & (point[1] <= e_max[:,1]) & (e_min[:,2]<= point[2]) & (point[2] <= e_max[:,2])]
    
    # It checks if the ttrd contains the point
    def check_ttrd(ttrd, point):
        coord = np.column_stack((ttrd[1,:]-ttrd[0,:],ttrd[2,:]-ttrd[0,:],ttrd[3,:]-ttrd[0,:]))
        coord = np.linalg.inv(coord).dot(point-ttrd[0,:])
        return coord.min() >= 0 and coord.sum() <= 1
    # It checks if the ttrd with num ttrdNum contains the point
    def check_ttrd_byNum(ttrdNum, point):
        ttrd = nodes[elems[ttrdNum]-1]
        return check_ttrd(ttrd,point)
    
    # Just takes all ttrds that contain points
    nodeIndices = elems[[x for x in potential_ttrds if check_ttrd_byNum(x,point)]][0]; 
    ns = nodes[nodeIndices-1]

    norms = np.sum((ns-point)**2,axis=-1)**0.5
    weights = 1/(norms+1e-10)
    weights = weights / weights.sum()
    
    return {"Nodes":nodeIndices,"Weights":weights}

In [15]:
# a function to get e-field vector at a given position [x,y,z]
def get_field(ttt, point, my_field):
    best_ttt=get_ttrd(ttt,point)
    return np.dot(my_field[best_ttt['Nodes']-1].T, best_ttt['Weights'])

In [16]:
# a function to calculate directional derivatives of the effective field at a given point [x,y,z]
def deriv_e_field(coordinates, e_field_nodes, LSD, ttt):
   
    step=0.05

    x1=coordinates[0]
    y1=coordinates[1]
    z1=coordinates[2]
    x0=coordinates[0]-step
    x2=coordinates[0]+step
    y0=coordinates[1]-step
    y2=coordinates[1]+step
    z0=coordinates[2]-step
    z2=coordinates[2]+step

    f_x0_y1_z1=np.dot(get_field(ttt,np.asarray([x0,y1,z1]), e_field_nodes), LSD)
    f_x2_y1_z1=np.dot(get_field(ttt,np.asarray([x2,y1,z1]), e_field_nodes), LSD)
    f_x1_y1_z1=np.dot(get_field(ttt,np.asarray([x1,y1,z1]), e_field_nodes), LSD)
    f_x1_y0_z1=np.dot(get_field(ttt,np.asarray([x1,y0,z1]), e_field_nodes), LSD)
    f_x1_y2_z1=np.dot(get_field(ttt,np.asarray([x1,y2,z1]), e_field_nodes), LSD)
    f_x1_y1_z0=np.dot(get_field(ttt,np.asarray([x1,y1,z0]), e_field_nodes), LSD)
    f_x1_y1_z2=np.dot(get_field(ttt,np.asarray([x1,y1,z2]), e_field_nodes), LSD)
    
    gradx=my_deriv(1,[f_x0_y1_z1,f_x1_y1_z1,f_x2_y1_z1], step)
    grady=my_deriv(1,[f_x1_y0_z1,f_x1_y1_z1,f_x1_y2_z1], step)
    gradz=my_deriv(1,[f_x1_y1_z0,f_x1_y1_z1,f_x1_y1_z2], step)
    
    return np.dot([gradx, grady, gradz], LSD)

In [64]:
# a function to compute the TMS effects for a given coil position and coil direction
def change_TMS_effects(x,y,z, x_dir, y_dir, z_dir, current_out_dir):
    
    l = 2 # membrane space constant 2mm
    l2 = l**2
    effect_max = -1000000

    position = [x - 256/2, y - 256/2, z - 256/2]
    direction = [x_dir - 256/2, y_dir - 256/2, z_dir - 256/2]
    # nonlocal current_out_dir
    # current_out_dir = out_dir + str(x) + '_' + str(y) + '_' + str(z)
    simulation(mesh_path, current_out_dir, pos_centre = position, pos_ydir = direction)
    mesh_file = current_out_dir + '/' + subject_name + '_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
    field_mesh = simnibs.msh.read_msh(mesh_file)
    field_as_nodedata = field_mesh.elmdata[0].as_nodedata()
    field_at_nodes = field_as_nodedata.value
    
    ttt = load_elems(field_mesh.nodes.node_coord, field_mesh.elm.node_number_list)
    
    effective_field=copy.deepcopy(new_streams_T1_array)
    
    for stream in range(len(new_streams_T1_array)):
        my_steam = copy.deepcopy(new_streams_T1_array[stream])
        # print ('starting _' + str(stream) + ' out of ' + str(len(new_streams_T1_array)))
        for t in range(len(my_steam[:, 0])):
            #-256/2 because of a freesurfer RAS coordinate system
            x = my_steam[t, 0] - 256/2
            y = my_steam[t, 1] - 256/2
            z = my_steam[t, 2] - 256/2
            xyz=np.asarray([x, y, z])

            field_vector_xyz = get_field(ttt, xyz, field_at_nodes)

            effective_field[stream][t,0] = l*np.dot(field_vector_xyz,streams_array_derivative[stream][t,:]) 
            effective_field[stream][t,1] = l2*deriv_e_field(xyz,field_at_nodes,streams_array_derivative[stream][t,:],ttt)
            effective_field[stream][t,2] = effective_field[stream][t,0]+effective_field[stream][t,1]
            if (effective_field[stream][t,2] > effect_max):
                effect_max=effective_field[stream][t,2]
            
    with open(current_out_dir + '/' + subject_name + '_effective_field_correct_12.txt', 'wb') as f:
        pickle.dump(effective_field, f)
    f.close()

    return effect_max

In [19]:
# create brain mask

from dipy.segment.mask import median_otsu

mask_data, mask = median_otsu(data_T1, 2, 1)

In [27]:
target_streams_number = np.shape(new_streams_T1_array)[0]
for target_s in range(1, target_streams_number):
    new_streams_T1_array[0] =np.concatenate((new_streams_T1_array[0], new_streams_T1_array[target_s]))

In [42]:
x_min = np.min(new_streams_T1_array[0][:,0]) - 25
x_max = np.max(new_streams_T1_array[0][:,0]) + 25
y_min = np.min(new_streams_T1_array[0][:,1]) - 25
y_max = np.max(new_streams_T1_array[0][:,1]) + 25
z_min = np.min(new_streams_T1_array[0][:,2]) - 25
z_max = np.max(new_streams_T1_array[0][:,2]) + 25
print (x_min, x_max, y_min, y_max, z_min, z_max)

x_min_init = np.min(new_streams_T1_array[0][:,0]) - 25
x_max_init = np.max(new_streams_T1_array[0][:,0]) + 25
y_min_init = np.min(new_streams_T1_array[0][:,1]) - 25
y_max_init = np.max(new_streams_T1_array[0][:,1]) + 25
z_min_init = np.min(new_streams_T1_array[0][:,2]) - 25
z_max_init = np.max(new_streams_T1_array[0][:,2]) + 25

68.90686061605352 148.95523340777123 79.0378371386976 153.0020098344636 86.8156621889938 221.97430390134028


In [65]:
number_of_trials = 25
number_in_trial = 10
n_best = 5
best_array = np.zeros((7, n_best))

In [57]:
import random
import math

def get_pt(x_lim1, x_lim2, y_lim1, y_lim2, z_lim1, z_lim2):
    
    point_ok = False
    
    while point_ok==False:
        px = random.uniform(x_lim1, x_lim2)
        py = random.uniform(y_lim1, y_lim2)
        pz = random.uniform(z_lim1, z_lim2)
        if mask[int(round(px)), int(round(py)), int(round(pz))] == False:
            point_ok = True
            
    angle = random.uniform(0, 2*math.pi)
    cx0 = px + math.cos(angle)
    cy0 = py + math.sin(angle)
    cz0 = pz
    
    center=256/2
    vector1 = [cx0 - px, cy0 - py, cz0 - pz]
    vector2 = [center - px, center - py, center - pz]
    cos_alpha = np.dot(vector1, vector2)
    sin_alpha = 1 - cos_alpha ** 2
    cz1 = cz0 + cos_alpha
    cx1 = px + sin_alpha * (cx0 - px)
    cy1 = py + sin_alpha * (cy0 - py)
    return px, py, pz, cx1, cy1, cz1

In [66]:
import shutil

for current_trial in range(number_of_trials):
    print ("start trial: ", current_trial, "out of ", number_of_trials)
    for current_point in range(number_in_trial):
        if current_point<number_in_trial*0.8:
            pt_x, pt_y, pt_z, cl_x, cl_y, cl_z = get_pt(x_min, x_max, y_min, y_max, z_min, z_max)
        else:
            pt_x, pt_y, pt_z, cl_x, cl_y, cl_z = get_pt(x_min_init, x_max_init, y_min_init, y_max_init, z_min_init, z_max_init)
        
        print ("start point: ", current_point, ":", pt_x, pt_y, pt_z)
        current_out_dir = out_dir + str(pt_x) + '_' + str(pt_y) + '_' + str(pt_z)
        current_effect = change_TMS_effects(pt_x, pt_y, pt_z, cl_x, cl_y, cl_z, current_out_dir)
        if current_effect > np.min(best_array[6, :]):
            print ("Better point was found!")
            ind_min = np.argmin(best_array[6, :])
            bad_folder = out_dir + str(best_array[0, ind_min]) + '_' + str(best_array[1, ind_min]) + '_' + str(best_array[2, ind_min])
            best_array[0, ind_min] = pt_x
            best_array[1, ind_min] = pt_y
            best_array[2, ind_min] = pt_z
            best_array[3, ind_min] = cl_x
            best_array[4, ind_min] = cl_y
            best_array[5, ind_min] = cl_z
            best_array[6, ind_min] = current_effect
            try:
                shutil.rmtree(bad_folder)
            except OSEerror as e:
                print ("Error: %s : %s" %(bad_folder, e.strerror))
        else:
            try:
                shutil.rmtree(current_out_dir)
            except OSEerror as e:
                print ("Error: %s : %s" %(current_out_dir, e.strerror))
    x_min = np.min(best_array[0, :])
    x_max = np.max(best_array[0, :])
    y_min = np.min(best_array[1, :])
    y_max = np.max(best_array[1, :])
    z_min = np.min(best_array[2, :])
    z_max = np.max(best_array[2, :])

best_coordinate=np.argmax(best_array[6, :])
print (best_array[:, best_coordinate])

[ simnibs ]WARNING: Cannot locate subjects m2m folder
[ simnibs ]WARNING: some postprocessing options might fail
[ simnibs ]INFO: Head Mesh: /home/sofya/Example_data/subject.msh
[ simnibs ]INFO: Subject Path: None
[ simnibs ]INFO: Simulation Folder: /home/sofya/Example_data/Output/simulation_at_pos_84.75146827764513_111.42393323127607_208.19982354005631


start trial:  0 out of  25
start point:  0 : 84.75146827764513 111.42393323127607 208.19982354005631


[ simnibs ]INFO: Running simulations in the directory: /home/sofya/Example_data/Output/simulation_at_pos_84.75146827764513_111.42393323127607_208.19982354005631
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/sofya/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[-1.2212e-01 -8.6722e-01  4.8271e-01 -4.1768e+01]
 [-9.8754e-01  1.5480e-01  2.8267e-02 -1.6723e+01]
 [-9.9239e-02 -4.7325e-01 -8.7532e-01  7.7330e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[ simnibs ]INFO: Assembling FEM Matrx
[ simnibs ]INFO: 20.10s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
[ simnibs ]INFO: Time to prepare KSP: 2.65s
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iter

Better point was found!
start point:  1 : 85.3471835380757 103.0308721855508 203.47817843196333


[ simnibs ]INFO: Running simulations in the directory: /home/sofya/Example_data/Output/simulation_at_pos_85.3471835380757_103.0308721855508_203.47817843196333
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/sofya/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[-8.4529e-01 -1.9488e-01  4.9750e-01 -4.2966e+01]
 [-1.5708e-01  9.8060e-01  1.1722e-01 -2.4697e+01]
 [-5.1069e-01  2.0932e-02 -8.5951e-01  7.6050e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[ simnibs ]INFO: Assembling FEM Matrx
[ simnibs ]INFO: 19.75s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
[ simnibs ]INFO: Time to prepare KSP: 2.69s
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterat

...

start point:  1 : 81.21373685445467 122.16659829357421 208.8761533189186


[ simnibs ]INFO: Running simulations in the directory: /home/sofya/Example_data/Output/simulation_at_pos_81.21373685445467_122.16659829357421_208.8761533189186
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/sofya/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[  0.6433   0.499    0.5806 -44.0713]
 [  0.664   -0.7412  -0.0987  -6.8148]
 [  0.3811   0.4491  -0.8082  75.6904]
 [  0.       0.       0.       1.    ]]
[ simnibs ]INFO: Assembling FEM Matrx
[ simnibs ]INFO: 19.83s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field
[ simnibs ]INFO: Time to prepare KSP: 2.61s
[ simnibs ]INFO: Running PETSc with KSP: cg PC: hypre
[ simnibs ]INFO: Number of iterations: 37 Residual Norm: 7.09e-11
[ simnibs ]INF

Better point was found!
start point:  2 : 80.00329659077961 123.67061740722347 207.95823057030603


[ simnibs ]INFO: Running simulations in the directory: /home/sofya/Example_data/Output/simulation_at_pos_80.00329659077961_123.67061740722347_207.95823057030603
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Coil file: /home/sofya/anaconda3/envs/simnibs_env/lib/python3.7/site-packages/simnibs/ccd-files/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities
[ simnibs ]INFO: Calculating Coil position from (centre, pos_y, distance)
[ simnibs ]INFO: Matsimnibs: 
[[ 8.2339e-01 -2.1956e-02  5.6705e-01 -4.4206e+01]
 [ 4.5130e-02 -9.9355e-01 -1.0400e-01 -5.2008e+00]
 [ 5.6568e-01  1.1123e-01 -8.1709e-01  7.5324e+01]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]
[ simnibs ]INFO: Assembling FEM Matrx
[ simnibs ]INFO: 19.66s to assemble FEM matrix
[ simnibs ]INFO: Calculating dA/dt field


SolverError: There was an error setting up the solver.
PETSc returned error code: 72463